<a href="https://colab.research.google.com/github/Jung0Jin/Pytorch_study/blob/master/9.%EC%88%9C%ED%99%98%EC%8B%A0%EA%B2%BD%EB%A7%9D(RecurrentNeuralNetwork).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

출처: https://wikidocs.net/60760

# 9. 순환 신경망(Recurrent Neural Network)

이번 챕터에서는 RNN(순환 신경망)에 대해서 이해하고, RNN의 장기 의존성 문제를 보완한 LSTM, 그리고 RNN과 LSTM을 이용하여 각 종 딥 러닝 실습을 진행해보겠습니다.

## 9.1. 순환 신경망(Recurrent Neural Network, RNN)

RNN(Recurrent Neural Network)은 시퀀스(Sequence) 모델입니다. 입력과 출력을 시퀀스 단위로 처리하는 모델입니다. 번역기를 생각해보면 입력은 번역하고자 하는 문장. 즉, 단어 시퀀스입니다. 출력에 해당되는 번역된 문장 또한 단어 시퀀스입니다. 이러한 시퀀스들을 처리하기 위해 고안된 모델들을 시퀀스 모델이라고 합니다. 그 중에서도 RNN은 딥 러닝에 있어 가장 기본적인 시퀀스 모델입니다.



```
용어는 비슷하지만 순환 신경망과 재귀 신경망(Recursive Neural Network)은 전혀 다른 개념입니다.
```



### 9.1.1. 순환 신경망(Recurrent Neural Network, RNN)

앞서 배운 신경망들은 전부 은닉층에서 활성화 함수를 지난 값은 오직 출력층 방향으로만 향했습니다. 이와 같은 신경망들을 피드 포워드 신경망(Feed Forward Neural Network)이라고 합니다. 그런데 그렇지 않은 신경망들이 있습니다. RNN(Recurrent Neural Network) 또한 그 중 하나입니다. RNN은 은닉층의 노드에서 활성화 함수를 통해 나온 결과값을 출력층 방향으로도 보내면서, 다시 은닉층 노드의 다음 계산의 입력으로 보내는 특징을 갖고 있습니다.

![](https://wikidocs.net/images/page/22886/rnn_image1_ver2.PNG)

이를 그림으로 표현하면 위와 같습니다. $x$는 입력층의 입력 벡터, $y$는 출력층의 출력 벡터입니다. 실제로는 편향 $b$도 입력으로 존재할 수 있지만 앞으로의 그림에서는 생략합니다. RNN에서 은닉층에서 활성화 함수를 통해 결과를 내보내는 역할을 하는 노드를 셀(cell)이라고 합니다. 이 셀은 이전의 값을 기억하려고 하는 일종의 메모리 역할을 수행하므로 이를 **메모리 셀** 또는 **RNN 셀**이라고 표현합니다.

은닉층의 메모리 셀은 각각의 시점(time step)에서 바로 이전 시점에서의 은닉층의 메모리 셀에서 나온 값을 자신의 입력으로 사용하는 재귀적 활동을 하고 있습니다. 앞으로는 현재 시점을 변수 t로 표현하겠습니다. 이는 현재 시점 t에서의 메모리 셀이 갖고 있는 값은 과거의 메모리 셀들의 값에 영향을 받은 것임을 의미합니다. 그렇다면 메모리 셀이 갖고 있는 이 값은 뭐라고 부를까요?

메모리 셀이 출력층 방향으로 또는 다음 시점 t+1의 자신에게 보내는 값을 **은닉 상태(hidden state)**라고 합니다. 다시 말해 t 시점의 메모리 셀은 t-1 시점의 메모리 셀이 보낸 은닉 상태값을 t 시점의 은닉 상태 계산을 위한 입력값으로 사용합니다.

![](https://wikidocs.net/images/page/22886/rnn_image2_ver3.PNG)

RNN을 표현할 때는 일반적으로 위의 그림에서 좌측과 같이 화살표로 사이클을 그려서 재귀 형태로 표현하기도 하지만, 우측과 같이 사이클을 그리는 화살표 대신 여러 시점으로 펼쳐서 표현하기도 합니다. 두 그림은 동일한 그림으로 단지 사이클을 그리는 화살표를 사용하여 표현하였느냐, 시점의 흐름에 따라서 표현하였느냐의 차이일 뿐 둘 다 동일한 RNN을 표현하고 있습니다.

피드 포워드 신경망에서는 뉴런이라는 단위를 사용했지만, RNN에서는 뉴런이라는 단위보다는 입력층과 출력층에서는 각각 입력 벡터와 출력 벡터, 은닉층에서는 은닉 상태라는 표현을 주로 사용합니다. 그래서 사실 위의 그림에서 회색과 초록색으로 표현한 각 네모들은 기본적으로 벡터 단위를 가정하고 있습니다. 피드 포워드 신경망과의 차이를 비교하기 위해서 RNN을 뉴런 단위로 시각화해보겠습니다.

![](https://wikidocs.net/images/page/22886/rnn_image2.5.PNG)

위의 그림은 입력 벡터의 차원이 4, 은닉 상태의 크기가 2, 출력층의 출력 벡터의 차원이 2인 RNN이 시점이 2일 때의 모습을 보여줍니다. 다시 말해 뉴런 단위로 해석하면 입력층의 뉴런 수는 4, 은닉층의 뉴런 수는 2, 출력층의 뉴런 수는 2입니다.

![](https://wikidocs.net/images/page/22886/rnn_image3_ver2.PNG)

RNN은 입력과 출력의 길이를 다르게 설계할 수 있으므로 다양한 용도로 사용할 수 있습니다. 위 그림은 입력과 출력의 길이에 따라서 달라지는 RNN의 다양한 형태를 보여줍니다. 위 구조가 자연어 처리에서 어떻게 사용될 수 있는지 예를 들어봅시다. RNN 셀의 각 시점별 입, 출력의 단위는 사용자가 정의하기 나름이지만 가장 보편적인 단위는 '단어 벡터'입니다.

예를 들어 하나의 입력에 대해서 여러 개의 출력(one-to-many)의 모델은 하나의 이미지 입력에 대해서 사진의 제목을 출력하는 이미지 캡셔닝(Image Captioning) 작업에 사용할 수 있습니다. 사진의 제목은 단어들의 나열이므로 시퀀스 출력입니다.

![](https://wikidocs.net/images/page/22886/rnn_image3.5.PNG)

또한 단어 시퀀스에 대해서 하나의 출력(many-to-one)을 하는 모델은 입력 문서가 긍정적인지 부정적인지를 판별하는 감성 분류(sentiment classification), 또는 메일이 정상 메일인지 스팸 메일인지 판별하는 스펨 메일 분류(spam detection)에 사용할 수 있습니다. 위 그림은 RNN으로 스팸 메일을 분류할 때의 아키텍처를 보여줍니다.

![](https://wikidocs.net/images/page/22886/rnn_image3.7.PNG)

다 대 다(many-to-many)의 모델의 경우에는 입력 문장으로부터 대답 문장을 출력하는 챗봇과 입력 문장으로부터 번역된 문장을 출력하는 번역기, 개체명 인식이나 품사 태깅과 같은 작업이 속합니다. 위 그림은 개체명 인식을 수행할 때의 RNN 아키텍처를 보여줍니다.

이제 RNN에 대한 수식을 정의해보겠습니다.

![](https://wikidocs.net/images/page/22886/rnn_image4_ver2.PNG)

현재 시점 t에서의 은닉 상태값을 $h_t$라고 정의하겠습니다. 은닉층의 메모리 셀은 $h_t$를 계산하기 위해서 총 두개의 가중치를 갖게 됩니다. 하나는 입력층에서 입력값을 위한 가중치 $W_x$이고, 하나는 이전 시점 t-1의 은닉 상태값인 $h_{t-1}$을 위한 가중치 $W_h$입니다.

이를 식으로 표현하면 다음과 같습니다.

은닉층: $h_t = tanh(W_xx_t + W_hh_{t-1} + b)$

출력층: $y_t = f(W_yh_t + b)$

단, $f$는 비선형 활성화 함수 중 하나.

RNN의 은닉층 연산을 벡터와 행렬 연산으로 이해할 수 있습니다. 자연어 처리에서 RNN의 입력 $x_t$는 대부분의 경우에서 단어 벡터로 간주할 수 있는데, 단어 벡터의 차원을 $d$라고 하고, 은닉 상태의 크기를 $D_h$라고 하였을 때 각 벡터와 행렬의 크기는 다음과 같습니다.

$x_t: (d \times 1)$

$W_x: (D_h \times d)$

$W_h: (D_h \times D_h)$

$h_{t-1}: (D_h \times 1)$

$b: (D_h \times 1)$

배치 크기가 1이고, $d$와 $D_h$ 두 값 모두를 4로 가정하였을 때, RNN의 은닉층 연산을 그림으로 표현하면 아래와 같습니다.


![](https://wikidocs.net/images/page/22886/rnn_images4-5.PNG)

이때 $h_t$를 계산하기 위한 활성화 함수로는 주로 하이퍼볼릭탄젠트 함수(tanh)가 사용되지만, ReLU로 바꿔 사용하는 시도도 있습니다. 

위의 식에서 각각의 가중치 $W_x, W_h, W_y$의 값은 모든 시점에서 값을 동일하게 공유합니다. 만약, 은닉층이 2개 이상일 경우에는 은닉층 2개의 가중치는 서로 다릅니다.

출력층은 결과값인 $y_t$를 계산하기 위한 활성화 함수로는 상황에 따라 다를텐데, 예를 들어서 이진 분류를 해야하는 경우라면 시그모이드 함수를 사용할 수 있고 다양한 카테고리 중에서 선택해야하는 문제라면 소프트맥스 함수를 사용하게 될 것입니다.

### 9.1.2. 파이썬으로 RNN 구현하기

직접 Numpy로 RNN 층을 구현해보겠습니다. 앞서 메모리 셀에서 은닉 상태를 계산하는 식을 다음과 같이 정의하였습니다.

$h_t = tanh(W_xX_t + W_th_{t-1} + b)$

실제 구현에 앞서 간단히 의사 코드(pseudocode)를 작성해보겠습니다.



```
# 아래의 코드는 의사 코드(pseudocode)로 실제 동작하는 코드가 아님.

hidden_state_t = 0 # 초기 은닉 상태를 0(벡터)로 초기화
for input_t in input_length: # 각 시점마다 입력을 받는다.
  output_t = tanh(input_t, hidden_state_t) # 각 시점에 대해서 입력과 은닉 상태를 가지고 연산
  hidden_state_t = output_t # 계산 결과는 현재 시점의 은닉 상태가 된다.
```

우선 t 시점의 은닉 상태를 hidden_state_t라는 변수로 선언하였고, 입력 데이터의 길이를 input_length로 선언하였습니다. 이 경우, 입력 데이터의 길이는 곧 총 시점의 수(timesteps)가 됩니다. 그리고 t 시점의 입력값을 input_t로 선언하였습니다. 각 메모리 셀은 각 시점마다 input_t와 hidden_state_t(이전 상태의 은닉 상태)를 입력으로 활성화 함수인 하이퍼볼릭탄젠트 함수를 통해 현 시점의 hidden_state_t를 계산합니다.

의사 코드를 통해 간단히 개념 정립을 해보았습니다. 이제 RNN 층을 실제 동작되는 코드로 구현해보겠습니다. 아래의 코드는 이해를 돕기 위해 (timesteps, input_size) 크기의 2D 텐서를 입력으로 받았다고 가정하였으나, 실제로 파이토치에서는 (batch_size, timesteps, input_size)의 크기의 3D 텐서를 입력으로 받는 것을 기억합시다.



In [31]:
import numpy as np

timesteps = 10 # 시점의 수. NLP에서는 보통 문장의 길이가 된다.
input_size = 4 # 입력의 차원. NLP에서는 보통 단어 벡터의 차원이 된다.
hidden_size = 8 # 은닉 상태의 크기. 메모리 셀의 용량이다.

inputs = np.random.random((timesteps, input_size)) # 입력에 해당되는 2D 텐서

hidden_state_t = np.zeros((hidden_size,)) # 초기 은닉 상태는 0(벡터)로 초기화
# 은닉 상태의 크기 hidden_size로 은닉 상태를 만듬.

우선 시점, 입력의 차원, 은닉 상태의 크기, 그리고 초기 은닉 상태를 정의하였습니다. 현재 초기 은닉 상태는 0의 값을 가지는 벡터로 초기화가 된 상태입니다. 초기 은닉 상태를 출력해보겠습니다.

In [32]:
print(hidden_state_t) # 8의 크기를 가지는 은닉 상태. 현재는 초기 은닉 상태로 모든 차원이 0의 값을 가짐.

[0. 0. 0. 0. 0. 0. 0. 0.]


은닉 상태의 크기를 8로 정의하였으므로 8의 차원을 가지는 0의 값으로 구성된 벡터가 출력됩니다. 이제 가중치와 편향을 정의합니다.

In [33]:
Wx = np.random.random((hidden_size, input_size)) # (8, 4)크기의 2D 텐서 생성. 입력에 대한 가중치.
Wh = np.random.random((hidden_size, hidden_size)) # (8, 8)크기의 2D 텐서 생성. 은닉 상태에 대한 가중치.
b = np.random.random((hidden_size,)) # (8,)크기의 1D 텐서 생성. 이 값은 편향(bias).

가중치와 편향을 각 크기에 맞게 정의하였습니다. 가중치와 편향의 크기를 출력해보겠습니다.

In [34]:
print(np.shape(Wx))
print(np.shape(Wh))
print(np.shape(b))

(8, 4)
(8, 8)
(8,)


각 가중치와 편향의 크기는 다음과 같습니다. Wx는 (은닉 상태의 크기 $\times$ 입력의 차원), Wh는 (은닉 상태의 크기 $\times$ 은닉 상태의 크기), b는 (은닉 상태의 크기)의 크기를 가집니다. 이제 모든 시점의 은닉 상태를 출력한다고 가정하고, RNN 층을 동작시켜봅시다.

In [35]:
total_hidden_states = []

# 메모리 셀 동작
for input_t in inputs: # 각 시점에 따라서 입력값이 입력됨.
  output_t = np.tanh(np.dot(Wx, input_t) + np.dot(Wh, hidden_state_t) + b) # Wx * Wh * Ht-1 + b(bias)
  total_hidden_states.append(list(output_t)) # 각 시점 t별 메모리 셀의 출력의 크기는 (timestep, output_dim)
  print(np.shape(total_hidden_states)) # 각 시점 t별 메모리 셀의 출력의 크기는 (timestep, output_dim)
  hidden_state_t = output_t

total_hidden_states = np.stack(total_hidden_states, axis=0)
# 출력 시 값을 깔끔하게 해준다.

print(total_hidden_states) # (timesteps, output_dim)의 크기. 이 경우 (10, 8)의 크기를 가지는 메모리 셀의 2D 텐서를 출력.

(1, 8)
(2, 8)
(3, 8)
(4, 8)
(5, 8)
(6, 8)
(7, 8)
(8, 8)
(9, 8)
(10, 8)
[[0.93565794 0.95098413 0.76839855 0.9741363  0.85391243 0.93436409
  0.89468524 0.82302962]
 [0.99928593 0.99971843 0.9999441  0.99997196 0.99991623 0.99997784
  0.9999478  0.99995174]
 [0.99977551 0.99983382 0.99998564 0.99998096 0.9999664  0.99997007
  0.9999605  0.99997383]
 [0.99988936 0.99982057 0.9999862  0.99998544 0.99998038 0.9999779
  0.99997135 0.99998673]
 [0.99978802 0.99993511 0.99998637 0.99999462 0.9999752  0.99999443
  0.99998529 0.99998785]
 [0.99993357 0.9999094  0.99998997 0.99999468 0.99999029 0.9999861
  0.99996852 0.99999156]
 [0.9999191  0.99991297 0.99999013 0.99999272 0.99998271 0.99998949
  0.99998592 0.99999027]
 [0.99982592 0.99987654 0.99998301 0.99999296 0.99998521 0.99998763
  0.99995646 0.99998851]
 [0.99992825 0.99989085 0.99998491 0.99999504 0.99998944 0.99999515
  0.99998828 0.99999563]
 [0.99993647 0.99995557 0.99999332 0.99999634 0.99998755 0.99999058
  0.99998295 0.99999026]]


### 9.1.3. 파이토치의 nn.RNN()

파이토치에서는 nn.RNN()을 통해서 RNN 셀을 구현합니다. 실습을 통해 이해해봅시다. 우선 필요한 파이토치의 도구들을 임포트합니다.

In [36]:
import torch
import torch.nn as nn

이제 입력의 크기와 은닉 상태의 크기를 정의합니다. 은닉 상태의 크기는 대표적인 RNN의 하이퍼파라미터입니다. 여기서 입력의 크기는 매 시점마다 들어가는 입력의 크기를 의미합니다.

In [37]:
input_size = 5 # 입력의 크기
hidden_size = 8 # 은닉 상태의 크기

이제 입력 텐서를 정의합니다. 입력 텐서는 (배치 크기 $\times$ 시점의 수 $\times$ 매 시점마다 들어가는 입력)의 크기를 가집니다. 여기서는 배치 크기는 1, 10번의 시점동안 5차원의 입력 벡터가 들어가도록 텐서를 정의합니다.

In [38]:
# (batch_size, time_steps, input_size)
inputs = torch.Tensor(1, 10, 5)

이제 nn.RNN()을 사용하여 RNN의 셀을 만듭니다. 인자로 입력의 크기, 은닉 상태의 크기를 정의해주고, batch_first=True를 통해서 입력 텐서의 첫번째 차원이 배치 크기임을 알려줍니다.

In [39]:
cell = nn.RNN(input_size, hidden_size, batch_first=True)

입력 텐서를 RNN 셀에 입력하여 출력을 확인해봅시다.

In [40]:
outputs, _status = cell(inputs)

RNN 셀은 두 개의 입력을 리턴하는데, 첫번째 리턴값은 모든 시점(timesteps)의 은닉 상태들이며, 두번째 리턴값은 마지막 시점(timestep)의 은닉 상태입니다. 우선 첫번째 리턴값에 대해서 크기를 확인해봅시다.

In [41]:
print(outputs.shape) # 모든 time-step의 hidden_state

torch.Size([1, 10, 8])


첫번째 리턴값의 은닉 상태들은 (1, 10, 8)의 크기를 가집니다. 이는 10번의 시점동안 8차원의 은닉 상태가 출력되었다는 의미입니다. 두번째 리턴값, 다시 말해 마지막 시점의 은닉 상태의 크기를 확인해보겠습니다.

In [42]:
print(_status.shape) # 최종 time-step의 hidden_state

torch.Size([1, 1, 8])


마지막 시점의 은닉 상태는 (1, 1, 8)의 크기를 가집니다.

### 9.1.4. 깊은 순환 신경망(Deep Recurrent Neural Network)

![](https://wikidocs.net/images/page/22886/rnn_image4.5_finalPNG.PNG)

앞서 RNN도 다수의 은닉층을 가질 수 있다고 언급한 바 있습니다. 위의 그림은 순환 신경망에서 은닉층이 1개 더 추가되어 은닉층이 2개인 깊은(deep) 순환 신경망의 모습을 보여줍니다. 위의 코드에서 첫번째 은닉층은 다음 은닉층에 모든 시점에 대해서 은닉 상태 값을 다음 은닉층으로 보내주고 있습니다.

깊은 순환 신경망을 파이토치로 구현할 때는 nn.RNN()의 인자인 num_layers에 값을 전달하여 층을 쌓습니다. 층이 2개인 깊은 순환 신경망의 경우, 앞서 실습했던 임의의 입력에 대해서 출력이 어떻게 달라지는지 확인해봅시다.

In [43]:
# (batch_size, time_steps, input_size)
inputs = torch.Tensor(1, 10, 5)

In [44]:
cell = nn.RNN(input_size = 5, hidden_size = 8, num_layers = 2, batch_first=True)

In [45]:
print(outputs.shape) # 모든 time-step의 hidden_state

torch.Size([1, 10, 8])


첫번째 리턴값의 크기는 층이 1개였던 RNN 셀 때와 달라지지 않습니다. 여기서는 마지막 층의 모든 시점의 은닉 상태들입니다.

In [46]:
print(_status.shape) # (층의 개수, 배치 크기, 은닉 상태의 크기)

torch.Size([1, 1, 8])


두번째 리턴값의 크기는 층이 1개였던 RNN 셀 때와 달라졌는데, 여기서 크기는 (층의 개수, 배치 크기, 은닉 상태의 크기)에 해당됩니다.

### 9.1.5. 양방향 순환 신경망(Bidirection Recurrent Neural Network)

양방향 순환 신경망은 시점 t에서의 출력값을 예측할 때 이전 시점의 데이터 뿐만 아니라, 이후 데이터로도 예측할 수 있다는 아이디어에 기반합니다.

영어 빈칸 채우기 문제에 비유하여 보겠습니다.



```
Exercise **is** very effective at [               ] belly fat.

1) reducing
2) increasing
3) multiplying
```

'운동은 복부 지방을 [     ] 효과적이다'라는 영어 문장이고, 정답은 reducing(줄이는 것)입니다. 그런데 위의 영어 빈 칸 채우기 문제를 잘 생각해보면 정답을 찾기 위해서는 이전에 나온 단어들만으로는 부족합니다. 목적어인 belly fat(복부 지방)를 모르는 상태라면 정답을 결정하기가 어렵습니다.

즉, RNN이 과거 시점(time step)의 데이터들을 참고해서, 찾고자하는 정답을 예측하지만 실제 문제에서는 과거 시점의 데이터만 고려하는 것이 아니라 향후 시점의 데이터에 힌트가 있는 경우도 많습니다. 그래서 이전 시점의 데이터뿐만 아니라, 이후 시점의 데이터도 힌트로 활용하기 위해서 고안된 것이 양방향 RNN입니다.


![](https://wikidocs.net/images/page/22886/rnn_image5_ver2.PNG)

양방향 RNN은 하나의 출력값을 예측하기 위해 기본적으로 두 개의 메모리 셀을 사용합니다. 첫번째 메모리 셀은 앞에서 배운 것처럼 **앞 시점의 은닉 상태(Forward States**)를 전달 받아 현재의 은닉 상태를 계산합니다. 위의 그림에서는 주황색 메모리 셀에 해당됩니다. 두번째 메모리 셀은 앞에서 배운 것과는 다릅니다. 앞 시점의 은닉 상태가 아니라 **뒤 시점의 은닉 상태(Backward States)**를 전달 받아 현재의 은닉 상태를 계산합니다. 위의 그림에서는 초록색 메모리 셀에 해당됩니다. 그리고 이 두 개의 값 모두가 출력층에서 출력값을 예측하기 위해 사용됩니다.

물론, 양방향 RNN도 다수의 은닉층을 가질 수 있습니다. 아래의 그림은 양방향 순환 신경망에서 은닉층이 1개 더 추가되어 은닉층이 2개인 깊은(deep) 양방향 순환 신경망의 모습을 보여줍니다.

![](https://wikidocs.net/images/page/22886/rnn_image6_ver3.PNG)

다른 인공 신경망 모델들도 마찬가지이지만, 은닉층을 무조건 추가한다고 해서 모델의 성능이 좋아지는 것은 아닙니다. 은닉층을 추가하면, 학습할 수 있는 양이 많아지지만 또한 반대로 훈련 데이터 또한 그만큼 많이 필요합니다.

양방향 순환 신경망을 파이토치로 구현할 때는 nn.RNN()의 인자인 bidirectional에 값을 True로 전달하면 됩니다. 이번에는 층이 2개인 깊은 순환 신경망이면서 양방향인 경우, 앞서 실습했던 임의의 입력에 대해서 출력이 어떻게 달라지는지 확인해봅시다.

In [47]:
# (batch_size, time_steps, input_size)
inputs = torch.Tensor(1, 10, 5)

In [48]:
cell = nn.RNN(input_size = 5, hidden_size = 8, num_layers = 2, batch_first = True, bidirectional = True)

In [49]:
outputs, _status = cell(inputs)

In [50]:
print(outputs.shape) # (배치 크기, 시퀀스 길이, 은닉 상태의 크기 x 2)

torch.Size([1, 10, 16])


첫번째 리턴값의 크기는 단방향 RNN 셀 때보다 은닉 상태의 크기의 값이 두 배가 되었습니다. 여기서는 (배치 크기, 시퀀스 길이, 은닉 상태의 크기 x 2)의 크기를 가집니다. 이는 양방향의 은닉 상태 값들이 연결(concatenate)되었기 때문입니다.

In [51]:
print(_status.shape) # (층의 개수 x 2, 배치 크기, 은닉 상태의 크기)

torch.Size([4, 1, 8])


두번째 리턴값의 크기는 (층의 개수 x 2, 배치 크기, 은닉 상태의 크기)를 가집니다. 이는 정방향 기준으로는 마지막 시점에 해당되면서, 역방향 기준에서는 첫번째 시점에 해당되는 시점의 출력값을 층의 개수만큼 쌓아 올린 결과값입니다.

## 9.2. 장단기 메모리(Long Short-Term Memory, LSTM)

바닐라 아이스크림이 가장 기본적인 맛을 가진 아이스크림인 것처럼, 앞서 배운 RNN을 가장 단순한 형태의 RNN이라고 하여 바닐라 RNN(Vanilla RNN)이라고 합니다. (케라스에서는 SimpleRNN) 바닐라 RNN 이후 바닐라 RNN의 한계를 극복하기 위한 다양한 RNN의 변형이 나왔습니다. 이번 챕터에서 배우게 될 LSTM도 그 중 하나입니다. 앞으로의 설명에서 LSTM과 비교하여 RNN을 언급하는 것은 전부 바닐라 RNN을 말합니다.

### 9.2.1. 바닐라 RNN의 한계

![](https://wikidocs.net/images/page/22888/lstm_image1_ver2.PNG)

앞 챕터에서 바닐라 RNN은 출력 결과가 이전의 계산 결과에 의존한다는 것을 언급한 바 있습니다. 하지만 바닐라 RNN은 비교적 짧은 시퀀스(sequence)에 대해서만 효과를 보이는 단점이 있습니다. 바닐라 RNN의 시점(time step)이 길어질수록 앞의 정보가 뒤로 충분히 전달되지 못하는 현상이 발생합니다. 위의 그림은 첫번째 입력값인 $x_1$의 정보량을 짙은 남색으로 표현했을 때, 색이 점차 얕아지는 것으로 시점이 지날수록 $x_1$의 정보량이 손실되어가는 과정을 표현하였습니다. 뒤로 갈수록 $x_1$의 정보량은 손실되고, 시점이 충분히 긴 상황에서는 $x_1$의 전체 정보에 대한 영향력은 거의 의미가 없을 수도 있습니다.

어쩌면 가장 중요한 정보가 시점의 앞 쪽에 위치할 수도 있습니다. RNN으로 만든 언어 모델이 다음 단어를 예측하는 과정을 생각해봅시다. 예를 들어 "모스크바에 여행을 왔는데 건물도 예쁘고 먹을 것도 맛있었어. 그런데 글쎄 직장 상사한테 전화가 왔어. 어디냐고 묻더라구 그래서 나는 말했지. 저 여행왔는데요. 여기 ___" 다음 단어를 예측하기 위해서는 장소 정보가 필요합니다. 그런데 장소 정보에 해당되는 단어인 '모스크바'는 앞에 위치하고 있고, RNN이 충분한 기억력을 가지고 있지 못한다면 다음 단어를 엉뚱하게 예측합니다.

이를 **장기 의존성 문제(the problem of Long-Term Dependencies)**라고 합니다.

### 9.2.2. 바닐라 RNN 내부 열어보기

![](https://wikidocs.net/images/page/22888/vanilla_rnn_ver2.PNG)

LSTM에 대해서 이해해보기 전에 바닐라 RNN의 뚜껑을 열어보겠습니다. 위의 그림은 바닐라 RNN의 내부 구조를 보여줍니다. 이 책에서는 RNN 계열의 인공 신경망의 그림에서는 편향 $b$를 생략합니다. 위의 그림에 편향 $b$를 그린다면 $x_t$ 옆에 tanh로 향하는 또 하나의 입력선을 그리면 됩니다.

$h_t = tanh(W_xx_t + W_hh_{t-1} + b)$

바닐라 RNN은 $x_t$와 $h_{t-1}$이라는 두 개의 입력이 각각의 가중치와 곱해져서 메모리 셀의 입력이 됩니다. 그리고 이를 하이퍼볼릭탄젠트 함수의 입력으로 사용하고 이 값은 은닉층의 출력인 은닉 상태가 됩니다.

### 9.2.3. LSTM(Long Short-Term Memory)

![](https://wikidocs.net/images/page/22888/vaniila_rnn_and_different_lstm_ver2.PNG)

위의 그림은 LSTM의 전체적인 내부의 모습을 보여줍니다. 전통적인 RNN의 이러한 단점을 보완한 RNN의 일종을 장단기 메모리(Long Short-Term Memory)라고 하며, 줄여서 LSTM이라고 합니다. LSTM은 은닉층의 메모리 셀에 입력 게이트, 망각 게이트, 출력 게이트를 추가하여 불필요한 기억을 지우고, 기억해야할 것들을 정합니다. 요약하면 LSTM은 은닉 상태(hidden state)를 계산하는 식이 전통적인 RNN보다 조금 더 복잡해졌으며 셀 상태(cell state)라는 값을 추가하였습니다. 위의 그림에서는 t시점의 셀 상태를 $C_t$로 표현하고 있습니다. LSTM은 RNN과 비교하여 긴 시퀀스의 입력을 처리하는데 탁월한 성능을 보입니다.

![](https://wikidocs.net/images/page/22888/cellstate.PNG)

셀 상태는 위의 그림에서 왼쪽에서 오른쪽으로 가는 굵은 선입니다. 셀 상태 또한 이전에 배운 은닉 상태처럼 이전 시점의 셀 상태가 다음 시점의 셀 상태를 구하기 위한 입력으로서 사용됩니다.

은닉 상태값과 셀 상태값을 구하기 위해서 새로 추가 된 3개의 게이트를 사용합니다. 각 게이트는 삭제 게이트, 입력 게이트, 출력 게이트라고 부르며 이 3개의 게이트에는 공통적으로 시그모이드 함수가 존재합니다. 시그모이드 함수를 지나면 0과 1사이의 값이 나오게 되는데 이 값들을 가지고 게이트를 조절합니다. 아래의 내용을 먼저 이해하고 각 게이트에 대해서 알아보도록 하겠습니다.



*   이하 식에서 $\sigma$는 시그모이드 함수를 의미합니다.
*   이하 식에서 tanh는 하이퍼볼릭탄젠트 함수를 의미합니다.
*   $W_{xi}, W_{xg}, W_{xf}, W_{xo}$는 $x_t$와 함께 각 게이트에서 사용되는 4개의 가중치입니다.
*   $W_{hi}, W_{hg}, W_{hf}, W_{ho}$는 $h_{t-1}$와 함께 각 게이트에서 사용되는 4개의 가중치입니다.
*   $b_i, b_g, b_f, b_o$는 각 게이트에서 사용되는 4개의 편향입니다.


**(1) 입력 게이트**


![](https://wikidocs.net/images/page/22888/inputgate.PNG)

$i_t = \sigma(W_{xi}x_t + W_{hi}h_{t-1} + b_i)$

$g_t = tanh(W_{xg}x_t + W_{hg}h_{t-1} + b_g)$

입력 게이트는 현재 정보를 기억하기 위한 게이트입니다. 우선 현재 시점 t의 $x$값과 입력 게이트로 이어지는 가중치 $W_{xi}$를 곱한 값과 이전 시점 t-1의 은닉 상태가 입력 게이트로 이어지는 가중치 $W_{hi}$를 곱한 값을 더하여 시그모이드 함수를 지납니다. 이를 $i_t$라고 합니다.

그리고 현재 시점 t의 $x$값과 입력 게이트로 이어지는 가중치 $W_{xi}$를 곱한 값과 이전 시점 t-1의 은닉 상태가 입력 게이트로 이어지는 가중치 $W_{hg}$를 곱한 값을 더하여 하이퍼볼릭탄젠트 함수를 지납니다. 이를 $g_t$라고 합니다.

시그모이드 함수를 지나 0과 1 사이의 값과 하이퍼볼릭탄젠트 함수를 지나 -1과 1 사이의 값 두 개가 나오게 됩니다. 이 두 개의 값을 가지고 이번에 선택된 기억할 정보의 양을 정하는데, 구체적으로 어떻게 결정하는지는 아래에서 배우게 될 셀 상태 수식을 보면 됩니다.

**(2) 삭제 게이트**

![](https://wikidocs.net/images/page/22888/forgetgate.PNG)

$f_t = \sigma(W_{xf}x_t + W_{hf}h_{t-1} + b_f)$

삭제 게이트는 기억을 삭제하기 위한 게이트입니다. 현재 시점 t의 $x$값과 이전 시점 t-1의 은닉 상태가 시그모이드 함수를 지나게 됩니다. 시그모이드 함수를 지나면 0과 1 사이의 값이 나오게 되는데, 이 값이 곧 삭제 과정을 거친 정보의 양입니다. 0에 가까울수록 정보가 많이 삭제된 것이고 1에 가까울수록 정보를 온전히 기억한 것입니다. 이를 가지고 셀 상태를 구하게 되는데, 구체적으로는 아래의 셀 상태 수식을 보면 됩니다.

**(3) 셀 상태(장기 상태)**

![](https://wikidocs.net/images/page/22888/cellstate2.PNG)

$C_t = f_t \circ C_{t-1} + i_t \circ g_t$

셀 상태 $C_t$를 LSTM에서는 장기 상태라고 부르기도 합니다. 그렇다면 셀 상태를 구하는 방법을 알아보겠습니다. 삭제 게이트에서 일부 기억을 잃은 상태입니다.

입력 게이트에서 구한 $i_t, g_t$ 이 두 개의 값에 대해서 원소별 곱(entrywise product)을 진행합니다. 다시 말해 같은 크기의 두 행렬이 있을 때 같은 위치의 성분끼리 곱하는 것을 말합니다. 여기서는 식으로 $\circ$ 로 표현합니다. 이것이 이번에 선택된 기억할 값입니다.

입력 게이트에서 선택된 기억을 삭제 게이트의 결과값과 더합니다. 이 값을 현재 시점 t의 셀 상태라고 하며, 이 값은 다음 t+1 시점의 LSTM 셀로 넘겨집니다.

삭제 게이트와 입력 게이트의 영향력을 이해해봅시다. 만약 삭제 게이트의 출력값인 $f_t$가 0이 된다면, 이전 시점의 셀 상태값인 $C_{t-1}$은 현재 시점의 셀 상태값을 결정하기 위한 영향력이 0이 되면서, 오직 입력 게이트의 결과만이 현재 시점의 셀 상태값 $C_t$을 결정할 수 있습니다. 이는 삭제 게이트가 완전히 닫히고 입력 게이트를 연 상태를 의미합니다. 반대로 입력 게이트의 $i_t$값을 0이라고 한다면, 현재 시점의 셀 상태값 $C_t$는 오직 이전 시점의 셀 상태값 $C_{t-1}$의 값에만 의존합니다. 이는 입력 게이트를 완전히 닫고 삭제 게이트만을 연 상태를 의미합니다. 결과적으로 삭제 게이트는 이전 시점의 입력을 얼마나 반영할지를 의미하고, 입력 게이트는 현재 시점의 입력을 얼마나 반영할지를 결정합니다.

**(4) 출력 게이트와 은닉 상태(단기 상태)**

![](https://wikidocs.net/images/page/22888/outputgateandhiddenstate.PNG)

$o_t = \sigma(W_{xo}x_t + W_{ho}h_{t-1} + b_o)$

$h_t = o_t \circ tanh(c_t)$

출력 게이트는 현재 시점 t의 $x$값과 이전 시점 t-1의 은닉 상태가 시그모이드 함수를 지난 값입니다. 해당 값은 현재 시점 t의 은닉 상태를 결정하는 일에 쓰이게 됩니다.

은닉 상태를 단기 상태라고 하기도 합니다. 은닉 상태는 장기 상태의 값이 하이퍼볼릭탄젠트 함수를 지나 -1과 1 사이의 값입니다. 해당 값은 출력 게이트의 값과 연산되면서, 값이 걸러지는 효과가 발생합니다. 단기 상태의 값은 또한 출력층으로도 향합니다.

### 9.2.4. 파이토치의 nn.LSTM()

파이토치에서 LSTM 셀을 사용하는 방법은 매우 간단합니다. 기존에 RNN 셀을 사용하려고 했을 때는 다음과 같이 사용했었습니다.



```
nn.RNN(input_dim, hidden_size, batch_first=True)
```

LSTM 셀은 이와 유사하게 다음과 같이 사용합니다.



```
nn.LSTM(input_dim, hidden_size, batch_first=True)
```




### 9.2.5. 파이토치 LSTM 예제

출처: http://cedartrees.co.kr/index.php/2020/08/01/pytorch-lstm-example/

이 예제는 파이토치를 활용해서 LSTM을 구현하는 예제입니다. LSTM은 RNN(Recurrent Neural Network)의 하나로 좋은 성능을 발휘하는 모델입니다. 파이토치는 LSTM을 직관적으로 구현할 수 있도록 하는 좋은 인공지능 프레임워크입니다.

본 예제는 현재 문장을 주고 다음 문장을 예측하는 알고리즘입니다. 이러한 예제들을 활용하면 다양한 시계열 데이터를 다룰 수 있습니다. 시계열 데이터라 함은 데이터가 어떤 시간의 순서를 가진다는 것을 의미합니다.

예를 들어 한 문장의 다양한 단어들은 비록 같은 단어일지라도 앞에 오느냐 뒤에 오느냐에 따라서 그 의미가 달라지는 경우가 있습니다. 그렇기 때문에 현재 문장을 유추하기 위해서는 앞에 어떤 단어가 있는지를 알아내는 것이 중요합니다. RNN은 이러한 예측을 가능하게 해줍니다.

RNN은 두 개의 Linear 모델이 합쳐진 하나의 Activation Function입니다. 아래 그림이 이를 잘 설명하고 있습니다.

![](https://i.imgur.com/Z2xbySO.png)

입력 문장은 "In the beginning God created the heavens and the earth"라는 문장입니다. x 데이터는 입력 문장이고 x 데이터를 통해 예측한 결과를 비교하기 위해서 정답 데이터셋 y를 만듭니다. y 데이터는 x 데이터의 첫번째 입력 'I'의 경우 'n'을 예측하고 'n'이 입력된 경우 ' '(공백) 을 시스템이 예측할 수 있도록 하기 위함입니다.

In [52]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

In [61]:
sentence = 'In the beginning God created the heavens and the earth'
x = sentence[:-1]
y = sentence[1:]

print(x)
print(y)

In the beginning God created the heavens and the eart
n the beginning God created the heavens and the earth


In [54]:
char_set = list(set(sentence))
input_size = len(char_set)
hidden_size = len(char_set)

print(char_set)
print(input_size)
print(hidden_size)

['t', 'a', 'b', 'o', 'i', 'r', 'v', ' ', 's', 'n', 'c', 'e', 'I', 'h', 'd', 'g', 'G']
17
17


index2char과 char2index는 각각 문자를 문자 자체로 입력하지 않고 one-hot의 형태로 입력하기 위해서 만들어준 python dict입니다. 

In [55]:
index2char = {i:c for i,c in enumerate(char_set)} # one-hot의 형태로 입력하기 위해서 만들어 준 dict
char2index = {c:i for i,c in enumerate(char_set)} # one-hot의 형태로 입력하기 위해서 만들어 준 dict

print(index2char)
print(char2index)

{0: 't', 1: 'a', 2: 'b', 3: 'o', 4: 'i', 5: 'r', 6: 'v', 7: ' ', 8: 's', 9: 'n', 10: 'c', 11: 'e', 12: 'I', 13: 'h', 14: 'd', 15: 'g', 16: 'G'}
{'t': 0, 'a': 1, 'b': 2, 'o': 3, 'i': 4, 'r': 5, 'v': 6, ' ': 7, 's': 8, 'n': 9, 'c': 10, 'e': 11, 'I': 12, 'h': 13, 'd': 14, 'g': 15, 'G': 16}


입력된 sentence는 그대로 입력값으로 사용하지 않고 one-hot 형태로 변경해서 최종 x_train 형태의 데이터를 만듭니다. 문장을 one-hot 형태로 만들기 위해서 numpyd의 eye함수를 사용합니다.

In [56]:
one_hot = []
for i, tkn in enumerate(x):
  one_hot.append(np.eye(len(char_set), dtype='int')[char2index[tkn]])

print(one_hot)

[array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0]), array([0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0]), array([0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]), array([1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]), array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0]), array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0]), array([0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]), array([0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]), array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0]), array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0]), array([0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]), array([0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0]), array([0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0]), array([0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]), array([0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0]), array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0]), array([0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,

In [57]:
x_train = torch.Tensor(one_hot)
x_train = x_train.view(1, len(x), -1)
print(x_train)

tensor([[[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
         [0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.

In [58]:
print(x_train.shape)

torch.Size([1, 53, 17])


x_train 데이터를 출력하면 위와 같은 형태가 만들어집니다. 위의 데이터셋은 [1, 53, 17] 형태의 3차원 데이터셋입니다. NLP 데이터는 3차원 형태의 입력 데이터셋을 가집니다. 첫번째 차원은 문장의 갯수, 두번째는 단어의 갯수, 세번째는 단어의 입력 차원입니다.

다음으로 아래와 같이 y_data를 만들어줍니다.

In [62]:
# y label

y_data = [char2index[c] for c in y]
y_data = torch.Tensor(y_data)

이제 모델을 만들 차례입니다. 파이토치는 nn.Module을 사용해서 Module을 만들 수 있습니다.

In [64]:
class RNN(nn.Module):

  # (batch_size, n, ) torch already know, you don't need to let torch know
  def __init__(self, input_size, hidden_size):
    super().__init__()
    self.input_size = input_size
    self.hidden_size = hidden_size

    self.rnn = nn.LSTM(
        input_size = input_size,
        hidden_size = hidden_size,
        num_layers = 4,
        batch_first = True,
        bidirectional = True
    )

    self.layers = nn.Sequential(
        nn.ReLU(),
        nn.Linear(input_size * 2, hidden_size),
    )

  def forward(self, x):
    y, _ = self.rnn(x)
    y = self.layers(y)
    return y

In [65]:
model = RNN(input_size, hidden_size)
model

RNN(
  (rnn): LSTM(17, 17, num_layers=4, batch_first=True, bidirectional=True)
  (layers): Sequential(
    (0): ReLU()
    (1): Linear(in_features=34, out_features=17, bias=True)
  )
)

RNN 클래스는 init, forward 함수로 구성됩니다. init함수는 LSTM 모델을 선언하는 부분과 softmax 함수를 선언하는 두 부분이 있습니다. LSTM 함수는 두 개의 인자값(input_size, hidden_size)을 기본으로 받습니다. input_size는 입력 벡터의 크기이며 hidden_size는 출력 벡터의 크기입니다. 본 예제에서는 입력 벡터와 출력 벡터의 크기가 같습니다. 배치 사이즈나 시퀀스 사이즈는 파이토치에서 자동으로 계산하기 때문에 입력할 필요가 없습니다. num_layers는 RNN의 층을 의미합니다. 본 예제는 4개의 층으로 구성했기 때문에 num_layers를 4로 설정했습니다. 그리고 bidirectional을 True로 했기 때문에 마지막 output의 형태는 input_size * 2의 형태가 됩니다. Linear 레이어는 input_size의 차원을 줄이기 위해서 선언합니다.

또 모델을 만들면서 중요한 것은 batch_first를 True로 해줘야 한다는 것입니다. 그렇지 않으면 time-step(=sequence_length), batch_size, input_vector의 형태가 됩니다.

In [66]:
# loss & optimizer setting
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters())

# start training
for i in range(5000):
  model.train()
  outputs = model(x_train)
  loss = criterion(outputs.view(-1, input_size), y_data.view(-1).long())

  optimizer.zero_grad()
  loss.backward()
  optimizer.step()

  if i % 500 == 0:
    result = outputs.data.numpy().argmax(axis=2)
    result_str = ''.join([char_set[c] for c in np.squeeze(result)])
    print(i, 'loss: ', loss.item(), '\nprediction: ', result, '\ntrue Y: ', y_data, '\nprediction str: ', result_str, '\n')

0 loss:  2.824496030807495 
prediction:  [[ 3 14 14 14 14 14 14 14 14 14 14 14 14 14 14 14 14 14 14 14 14 14 14 14
  14 14 14 14 14 14 14 14 14 14 14 14 14 14 14 14 14 14 14 14 14 14 14 14
  14 14 14 14 14]] 
true Y:  tensor([ 9.,  7.,  0., 13., 11.,  7.,  2., 11., 15.,  4.,  9.,  9.,  4.,  9.,
        15.,  7., 16.,  3., 14.,  7., 10.,  5., 11.,  1.,  0., 11., 14.,  7.,
         0., 13., 11.,  7., 13., 11.,  1.,  6., 11.,  9.,  8.,  7.,  1.,  9.,
        14.,  7.,  0., 13., 11.,  7., 11.,  1.,  5.,  0., 13.]) 
prediction str:  odddddddddddddddddddddddddddddddddddddddddddddddddddd 

500 loss:  1.6518795490264893 
prediction:  [[ 9  7 13 13 11 11 11 11 15  9  9  9  9  9  9 15  7  7  7  7  7  7 11 11
  11 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11
  11  1  5  0 13]] 
true Y:  tensor([ 9.,  7.,  0., 13., 11.,  7.,  2., 11., 15.,  4.,  9.,  9.,  4.,  9.,
        15.,  7., 16.,  3., 14.,  7., 10.,  5., 11.,  1.,  0., 11., 14.,  7.,
         0., 13., 11.,  7., 13., 

위의 코드와 같이 학습을 수행합니다. x_train 데이터를 입력 받아 나온 결과를 y_data와 비교해서 loss를 계산하고 이 loss 값을 Backpropagation을 수행하고 Gradient를 초기화하는 과정을 반복합니다.

### 9.2.6. Pytorch로 RNN, LSTM 구현하기

출처: https://justkode.kr/deep-learning/pytorch-rnn

RNN(Recurrent Neural Network)을 위한 API는 torch.nn.RNN(*args, **kwargs) 입니다.

일단 Input 시퀀스의 각 요소에 대해, 각 레이어에서는 다음 연산을 수행합니다.

$h_t = tanh(W_{ihx_t} + b_{ih} + W_{hh}h_{t-1} + b_{hh})$

Parameters



*   input_size: Input의 사이즈에 해당하는 수를 입력하면 됩니다.
*   hidden_size: 은닉층의 사이즈에 해당하는 수를 입력하면 됩니다.
*   num_layers: RNN의 은닉층 레어이 갯수를 나타냅니다. 기본 값은 1입니다.
*   nonlinearity: 비선형 활성화 함수를 선택합니다. tanh, relu 중 하나를 선택 가능하며, 기본 값은 tanh 입니다.
*   bias: 바이어스 값 활성화 여부를 선택합니다. 기본 값은 True입니다.
*   batch_first: True 일 시, Output 값의 사이즈는 (batch, seq, feature) 가 됩니다. 기본 값은 False 입니다.
*   dropout: 드롭아웃 비율을 설정합니다. 기본 값은 0 입니다.
*   bidirectional: True 일 시, 양방향 RNN이 됩니다. 기본 값은 False 입니다.

num_layers 가 2라면? 아래와 같은 그림의 신경망이 만들어진다고 생각하면 됩니다.



![](https://discuss.pytorch.org/uploads/default/optimized/2X/f/fb98eb0d16b722e019db59f97825aa529cb6bc08_2_685x499.png)

In [67]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim

In [68]:
sentences = ['i like dog',
             'i love coffee', 
             'i hate milk', 
             'you like cat',
             'you love milk',
             'you hate coffee']
dtype = torch.float

In [69]:
"""
Word Processing
"""

word_list = list(set(' '.join(sentences).split()))
word_dict = {w:i for i, w in enumerate(word_list)}
number_dict = {i:w for i, w in enumerate(word_list)}
n_class = len(word_dict)

print(word_list)
print(word_dict)
print(number_dict)
print(n_class)

['i', 'milk', 'hate', 'love', 'dog', 'you', 'like', 'coffee', 'cat']
{'i': 0, 'milk': 1, 'hate': 2, 'love': 3, 'dog': 4, 'you': 5, 'like': 6, 'coffee': 7, 'cat': 8}
{0: 'i', 1: 'milk', 2: 'hate', 3: 'love', 4: 'dog', 5: 'you', 6: 'like', 7: 'coffee', 8: 'cat'}
9


In [72]:
"""
TextRNN Parameter
"""

batch_size = len(sentences)
n_step = 2 # 학습하려고 하는 문장의 길이 - 1
n_hidden = 5 # 은닉층 사이즈

def make_batch(sentences):
  input_batch = []
  target_batch = []

  for sen in sentences:
    word = sen.split()
    input = [word_dict[n] for n in word[:-1]]
    target = word_dict[word[-1]]

    input_batch.append(np.eye(n_class)[input]) # One-Hot Encoding
    target_batch.append(target)

  return input_batch, target_batch

input_batch, target_batch = make_batch(sentences)
input_batch = torch.tensor(input_batch, dtype=torch.float32, requires_grad=True)
target_batch = torch.tensor(target_batch, dtype=torch.int64)

print(input_batch)
print(target_batch)

tensor([[[1., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 1., 0., 0.]],

        [[1., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 1., 0., 0., 0., 0., 0.]],

        [[1., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 1., 0., 0., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0., 1., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 1., 0., 0.]],

        [[0., 0., 0., 0., 0., 1., 0., 0., 0.],
         [0., 0., 0., 1., 0., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0., 1., 0., 0., 0.],
         [0., 0., 1., 0., 0., 0., 0., 0., 0.]]], requires_grad=True)
tensor([4, 7, 1, 8, 1, 7])


In [77]:
"""
TextRNN
"""

class TextRNN(nn.Module):
  def __init__(self):
    super(TextRNN, self).__init__()

    self.rnn = nn.RNN(input_size=n_class, hidden_size=n_hidden, dropout=0.3)
    self.W = nn.Parameter(torch.randn([n_hidden, n_class]).type(dtype))
    self.b = nn.Parameter(torch.randn([n_class]).type(dtype))
    self.Softmax = nn.Softmax(dim=1)

  def forward(self, hidden, X):
    X = X.transpose(0, 1)
    outputs, hidden = self.rnn(X, hidden)
    outputs = outputs[-1]  # 최종 예측 Hidden Layer
    model = torch.mm(outputs, self.W) + self.b  # 최종 예측 최종 출력 층
    return model

In [78]:
"""
Training
"""

model = TextRNN()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

for epoch in range(500):
  hidden = torch.zeros(1, batch_size, n_hidden, requires_grad=True)
  output = model(hidden, input_batch)
  loss = criterion(output, target_batch)

  if (epoch + 1) % 100 == 0:
    print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.6f}'.format(loss))
  
  optimizer.zero_grad()
  loss.backward()
  optimizer.step()

input = [sen.split()[:2] for sen in sentences]

hidden = torch.zeros(1, batch_size, n_hidden, requires_grad=True)
predict = model(hidden, input_batch).data.max(1, keepdim=True)[1]
print([sen.split()[:2] for sen in sentences], '->', [number_dict[n.item()] for n in predict.squeeze()])


/usr/local/lib/python3.6/dist-packages/torch/nn/modules/rnn.py:61: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.3 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


Epoch: 0100 cost = 0.536698
Epoch: 0200 cost = 0.174012
Epoch: 0300 cost = 0.031477
Epoch: 0400 cost = 0.014441
Epoch: 0500 cost = 0.008781
[['i', 'like'], ['i', 'love'], ['i', 'hate'], ['you', 'like'], ['you', 'love'], ['you', 'hate']] -> ['dog', 'coffee', 'milk', 'cat', 'milk', 'coffee']


LSTM(Long Short-Term Memory)를 위한 API는 torch.nn.LSTM(*args, **kwargs)입니다. LSTM은 기울기 폭발, 기울기 소실 등의 문제를 해결하기 위해 기존 RNN을 개선한 구조로, Input 시퀀스의 각 요소에 대해, 각 레이어에서는 다음 연산을 수행합니다.

$i_t = \sigma(W_{ii}x_t + b_{ii} + W_{hi}h_{t-1} + b_{hi})$

$f_t = \sigma(W_{if}x_t + b_{if} + W_{hf}h_{t-1} + b_{hf})$

$g_t = tanh(W_{ig}x_t + b_{ig} + W_{hg}h_{t-1} + b_{hg})$

$o_t = \sigma(W_{io}x_t + b_{io} + W_{ho}h_{t-1} + b_{ho})$

$c_t = f_t \circ c_{t-1} + i_t \circ g_t$

$h_t = o_t \circ tanh(c_t)$

Parameters (RNN과 비슷합니다.)



*   input_size: Input의 사이즈에 해당하는 수를 입력하면 됩니다.
*   hidden_size: 은닉층의 사이즈에 해당하는 수를 입력하면 됩니다.
*   num_layers: RNN의 은닉층 레이어 갯수를 나타냅니다. 기본 값은 1입니다.
*   bias: 바이어스 값 활성화 여부를 선택합니다. 기본 값은 True입니다.
*   batch_first: True일 시, Output 값의 사이즈는 (batch, seq, feature)가 됩니다. 기본 값은 False입니다.
*   dropout: 드롭아웃 비율을 설정합니다. 기본 값은 0입니다.
*   bidirectional: True일 시, 양방향 RNN이 됩니다. 기본 값은 False입니다.



In [80]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim

In [85]:
sentences = ["i like dog", 
             "i love coffee", 
             "i hate milk", 
             "you like cat", 
             "you love milk", 
             "you hate coffee"]
dtype = torch.float

In [86]:
"""
Word Processing
"""

word_list = list(set(" ".join(sentences).split()))
word_dict = {w: i for i, w in enumerate(word_list)}
number_dict = {i: w for i, w in enumerate(word_list)}
n_class = len(word_dict)

print(word_list)
print(word_dict)
print(number_dict)
print(n_class)

['i', 'milk', 'hate', 'love', 'dog', 'you', 'like', 'coffee', 'cat']
{'i': 0, 'milk': 1, 'hate': 2, 'love': 3, 'dog': 4, 'you': 5, 'like': 6, 'coffee': 7, 'cat': 8}
{0: 'i', 1: 'milk', 2: 'hate', 3: 'love', 4: 'dog', 5: 'you', 6: 'like', 7: 'coffee', 8: 'cat'}
9


In [87]:
"""
TextRNN Parameter
"""

batch_size = len(sentences)
n_step = 2  # 학습 하려고 하는 문장의 길이 - 1
n_hidden = 5  # 은닉층 사이즈

def make_batch(sentences):
  input_batch = []
  target_batch = []

  for sen in sentences:
    word = sen.split()
    input = [word_dict[n] for n in word[:-1]]
    target = word_dict[word[-1]]

    input_batch.append(np.eye(n_class)[input])  # One-Hot Encoding
    target_batch.append(target)
  
  return input_batch, target_batch

input_batch, target_batch = make_batch(sentences)
input_batch = torch.tensor(input_batch, dtype=torch.float32, requires_grad=True)
target_batch = torch.tensor(target_batch, dtype=torch.int64)

print(input_batch)
print(target_batch)

tensor([[[1., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 1., 0., 0.]],

        [[1., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 1., 0., 0., 0., 0., 0.]],

        [[1., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 1., 0., 0., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0., 1., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 1., 0., 0.]],

        [[0., 0., 0., 0., 0., 1., 0., 0., 0.],
         [0., 0., 0., 1., 0., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0., 1., 0., 0., 0.],
         [0., 0., 1., 0., 0., 0., 0., 0., 0.]]], requires_grad=True)
tensor([4, 7, 1, 8, 1, 7])


In [88]:
"""
TextLSTM
"""

class TextLSTM(nn.Module):
  def __init__(self):
    super(TextLSTM, self).__init__()

    self.lstm = nn.LSTM(input_size=n_class, hidden_size=n_hidden, dropout=0.3)
    self.W = nn.Parameter(torch.randn([n_hidden, n_class]).type(dtype))
    self.b = nn.Parameter(torch.randn([n_class]).type(dtype))
    self.Softmax = nn.Softmax(dim=1)

  def forward(self, hidden_and_cell, X):
    X = X.transpose(0, 1)
    outputs, hidden = self.lstm(X, hidden_and_cell)
    outputs = outputs[-1]  # 최종 예측 Hidden Layer
    model = torch.mm(outputs, self.W) + self.b  # 최종 예측 최종 출력 층
    return model

In [89]:
"""
Training
"""

model = TextLSTM()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

for epoch in range(500):
  hidden = torch.zeros(1, batch_size, n_hidden, requires_grad=True)
  cell = torch.zeros(1, batch_size, n_hidden, requires_grad=True)
  output = model((hidden, cell), input_batch)
  loss = criterion(output, target_batch)

  if (epoch + 1) % 100 == 0:
    print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.6f}'.format(loss))
  
  optimizer.zero_grad()
  loss.backward()
  optimizer.step()

input = [sen.split()[:2] for sen in sentences]

hidden = torch.zeros(1, batch_size, n_hidden, requires_grad=True)
cell = torch.zeros(1, batch_size, n_hidden, requires_grad=True)
predict = model((hidden, cell), input_batch).data.max(1, keepdim=True)[1]
print([sen.split()[:2] for sen in sentences], '->', [number_dict[n.item()] for n in predict.squeeze()])

/usr/local/lib/python3.6/dist-packages/torch/nn/modules/rnn.py:61: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.3 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


Epoch: 0100 cost = 0.209594
Epoch: 0200 cost = 0.024113
Epoch: 0300 cost = 0.010637
Epoch: 0400 cost = 0.006304
Epoch: 0500 cost = 0.004269
[['i', 'like'], ['i', 'love'], ['i', 'hate'], ['you', 'like'], ['you', 'love'], ['you', 'hate']] -> ['dog', 'coffee', 'milk', 'cat', 'milk', 'coffee']
